### The approach here is to distinguish the variants in multi-variant case. 
1. Use my defination to order the variant type
2. Find the coresponding info such as AF, allele sequencing, allele_conut, etc.
3. Use a datafame to store the informaiton and to munipulate
4. Use "requests' to query external API-ExAC for population informaiton such as allele frequency.

In [ ]:
#import all the package
import vcf
import allel
import h5py
import numpy as np
import pandas as pd
import requests
import pprint 
pd.set_option('display.max_columns', None)

#Load the file
filename = 'challenge_data.vcf'
#Save it to a npz file ehich is easier for me to process
allel.vcf_to_npz(filename, 'example.npz', fields='*', overwrite=True,numbers={'GT': 3})

In [ ]:
#Letus take a look what is in there. Wee need to pay attention to the GT number and sample number
callset = np.load('example.npz')

#how many variants a sample can have. We set this number as the max 
max_alt = np.max(callset['variants/numalt'])
#Transform these data into a dataframe called "data"
data = allel.vcf_to_dataframe(filename, fields='*', alt_number=6)

In [137]:
# A example
data[0:1]

,CHROM,POS,ID,REF,ALT_1,ALT_2,ALT_3,ALT_4,ALT_5,ALT_6,QUAL,NS,DP,DPB,AC_1,AC_2,AC_3,AC_4,AC_5,AC_6,AN,AF_1,AF_2,AF_3,AF_4,AF_5,AF_6,RO,AO_1,AO_2,AO_3,AO_4,AO_5,AO_6,PRO,PAO_1,PAO_2,PAO_3,PAO_4,PAO_5,PAO_6,QR,QA_1,QA_2,QA_3,QA_4,QA_5,QA_6,PQR,PQA_1,PQA_2,PQA_3,PQA_4,PQA_5,PQA_6,SRF,SRR,SAF_1,SAF_2,SAF_3,SAF_4,SAF_5,SAF_6,SAR_1,SAR_2,SAR_3,SAR_4,SAR_5,SAR_6,SRP,SAP_1,SAP_2,SAP_3,SAP_4,SAP_5,SAP_6,AB_1,AB_2,AB_3,AB_4,AB_5,AB_6,ABP_1,ABP_2,ABP_3,ABP_4,ABP_5,ABP_6,RUN_1,RUN_2,RUN_3,RUN_4,RUN_5,RUN_6,RPP_1,RPP_2,RPP_3,RPP_4,RPP_5,RPP_6,RPPR,RPL_1,RPL_2,RPL_3,RPL_4,RPL_5,RPL_6,RPR_1,RPR_2,RPR_3,RPR_4,RPR_5,RPR_6,EPP_1,EPP_2,EPP_3,EPP_4,EPP_5,EPP_6,EPPR,DPRA_1,DPRA_2,DPRA_3,DPRA_4,DPRA_5,DPRA_6,ODDS,GTI,TYPE_1,TYPE_2,TYPE_3,TYPE_4,TYPE_5,TYPE_6,CIGAR_1,CIGAR_2,CIGAR_3,CIGAR_4,CIGAR_5,CIGAR_6,NUMALT,MEANALT_1,MEANALT_2,MEANALT_3,MEANALT_4,MEANALT_5,MEANALT_6,LEN_1,LEN_2,LEN_3,LEN_4,LEN_5,LEN_6,MQM_1,MQM_2,MQM_3,MQM_4,MQM_5,MQM_6,MQMR,PAIRED_1,PAIRED_2,PAIRED_3,PAIRED_4,PAIRED_5,PAIRED_6,PAIREDR,MIN_DP,END,FILTER_PASS,numalt,altlen_1,altlen_2,altlen_3,altlen_4,altlen_5,altlen_6,is_snp
0,1,931393,.,G,T,NaN,NaN,NaN,NaN,NaN,2.179380e-13,2,4124,4124.0,0,-1,-1,-1,-1,-1,6,0.0,NaN,NaN,NaN,NaN,NaN,4029,95,-1,-1,-1,-1,-1,0.0,0.0,NaN,NaN,NaN,NaN,NaN,160284,3774,-1,-1,-1,-1,-1,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1663,2366,40,-1,-1,-1,-1,-1,55,-1,-1,-1,-1,-1,269.368988,8.15326,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1,-1,-1,-1,-1,-1,4.13032,NaN,NaN,NaN,NaN,NaN,101.278,51.0,NaN,NaN,NaN,NaN,NaN,44.0,NaN,NaN,NaN,NaN,NaN,9.61615,NaN,NaN,NaN,NaN,NaN,316.776001,0.999031,NaN,NaN,NaN,NaN,NaN,591.289978,0,snp,NaN,NaN,NaN,NaN,NaN,1X,NaN,NaN,NaN,NaN,NaN,1,1.0,NaN,NaN,NaN,NaN,NaN,1,-1,-1,-1,-1,-1,59.7579,NaN,NaN,NaN,NaN,NaN,65.227402,0.989474,NaN,NaN,NaN,NaN,NaN,0.966741,-1,-1,False,1,0,0,0,0,0,0,True


In [166]:

def variant_summary( df=data):
'''
1. Here. I made a function that take the dataframe as input and extract the following information
    'variant_type'         ---To answer Question 1:Type of variation
    'variant_allele',      ---What is the variant basepair
    'variant_allele_count' ---To answer Question 3:Number of reads supporting the variant
    'variant_AF':          ---Variant allele frequency
    'variant_vs_ref%'      ---Percentage of reads supporting the variant versus those supporting reference reads
'''
    #make an empty df
    summary = pd.DataFrame(columns=['variant_type',
                                    'variant_allele', 
                                    'variant_allele_count', 
                                    'variant_AF',
                                    'variant_vs_ref%']) 
       
      
    
    ref_column_index = df.columns.get_loc("REF") # find the reference column from data indexing
    ref_read_index = df.columns.get_loc("RO")# find the Reference allele observation count coloumn from data indexing
    AF_index =  df.columns.get_loc("AF_1")# find the variant allele observation countcoloumn from data index
    
    
    total_row = df.shape[0] # this is the total case number we need to process
    #========================================================================
    for row in range(total_row): # run each case
        case_call = []
        ref_allele = df.iloc[row]['REF'] # the REF allele sequence
        
        # make a list of variant_type, number of read, and AF for each variant    
        for number in range(df.iloc[row]['numalt']): #how many different vatiants this row has
            
            #Here we go through every different variants
            variant_allele = df.iloc[row, ref_column_index+number +1] # tge n-th variant allele
            variant_reading = df.iloc[row, ref_read_index +number +1] # match this variant to its allele read count
            variant_frequency = df.iloc[row, AF_index+number] # match variant to its AF
            variant_vs_ref = round((variant_reading/df.iloc[row]['RO']*100),2) # the % of variant vs ref count
            
            #Use a list to collect the information of one variant in this row 
            call = [variant_allele,variant_reading, variant_frequency, variant_vs_ref]
            #Use allel length to determine its type            
            if len(ref_allele) - len(variant_allele)== 0: #when ref and variant have the same length
                call.insert(0, 'Subtitution')
            if len(ref_allele) - len(variant_allele) < 0: #when ref is shorter than variant 
                call.insert(0, 'Insertion')
            if len(ref_allele) - len(variant_allele) > 0: # When ref is longer than variant  
                call.insert(0, 'Deletion')
            # this is a list of list contains all the variant info in this row
            case_call.append(call) 
    #=======================================================================
    '''
    2. The second step is to analyze the case that has single or multiple variants. 
        !!! The Task says "If there are multiple possibilities, annotate with the most deleterious possibility." 
        Quantify the level of varient deleteriousness is a big challenge in cancer genetics, which will take a few more decades.
        With my 15+ year of experience in cancer genetics and cancer biology, I simply cannot tell variant A is more deleterious
        than variant B without the meeting the patients in clinic and seeing their medical record.

        So I will simplify the task and artificially define deleteriousness as Deletion > Insertion > Substitution 

        To determine whether the variant is Silent, it require a more complex algorithm that consider the coding frame, 
        overlaping genes, genes on the reverse allele, etc. SO I will not put this catergory here.
    '''            
       # sorted the variant types by s dummp order
        sort_order = {"Deletion":0, "Insertion":1, "Subtitution":2, "SNP":3}
        # In this list of list, the first item is always the the variant_type. We use it as a sorting role
        case_call.sort(key=lambda val:sort_order[val[0]])
        final_report = (case_call[0]) # Make a  most "deleterious" as the final call this this row
        
        
        summary.loc[len(summary)]= final_report

    return summary

In [167]:
report = variant_summary() 
# Just make a copy with better name. This df now has five columns

C:\Users\ywang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:31: RuntimeWarning: divide by zero encountered in longlong_scalars


In [168]:
#Let's add a few more coulmns here with useful info
report.insert(1,'ref_allele', data['REF'])       # The ref allele sequence
report.insert(3,'ref_allele_count', data['RO'])  # ref allele count number
report['DP'] = data['DP']                        # Answer Question 3: Depth of sequence coverage at the site of variation.
report.insert(1,'CHROM', data['CHROM'])          # The location of variant = Chromosome number and position
report.insert(2,'POS', data['POS'])

In [187]:
#Let's make a string for query on the ExAC API
report['Query_name'] =   report['CHROM'] +'-' 
                       + report['POS'].astype(str)+ '-' 
                       + report['ref_allele'] + '-' 
                       + report['variant_allele']

In [ ]:
report['AF_ExAC'] = np.nan

In [205]:
# Take a look the df by far
report[0:10]

,variant_type,CHROM,POS,ref_allele,variant_allele,ref_allele_count,variant_allele_count,variant_AF,variant_vs_ref%,DP,Query_name,AF_ExAC
0,Subtitution,1,931393,G,T,4029,95,0.000000,2.360000,4124,1-931393-G-T,NaN
1,Subtitution,1,935222,C,A,480,652,0.666667,135.830000,1134,1-935222-C-A,NaN
2,Subtitution,1,1277533,T,C,0,786,1.000000,inf,786,1-1277533-T-C,NaN
3,Subtitution,1,1284490,G,A,0,228,1.000000,inf,228,1-1284490-G-A,NaN
4,Subtitution,1,1571850,G,A,3961,94,0.000000,2.370000,4055,1-1571850-G-A,NaN
5,Subtitution,1,1572579,A,G,3430,26,0.000000,0.760000,3456,1-1572579-A-G,NaN
6,Subtitution,1,1575616,T,C,0,1118,1.000000,inf,1118,1-1575616-T-C,NaN
7,Subtitution,1,1575784,C,T,3206,266,0.333333,8.300000,3476,1-1575784-C-T,NaN
8,Subtitution,1,1577180,C,T,40,50,0.666667,125.000000,90,1-1577180-C-T,NaN
9,Subtitution,1,1635004,T,C,0,4076,1.000000,inf,4076,1-1635004-T-C,NaN


In [261]:
#Let's test the ExAC API
query_name = 'http://exac.hms.harvard.edu/rest/variant/variant/'+report.loc[8]['Query_name']
r = requests.get(query_name)
#Use pprint to show a redable version of the query result
pprint.pprint(r.json())

{'allele_count': 42,
 'allele_freq': 0.4883720930232558,
 'allele_num': 86,
 'alt': 'T',
 'chrom': '1',
 'filter': 'VQSRTrancheSNP99.80to99.90',
 'genes': ['ENSG00000248333'],
 'genotype_depths': [[[2.5, 1801],
                      [7.5, 271],
                      [12.5, 40],
                      [17.5, 5],
                      [22.5, 1],
                      [27.5, 0],
                      [32.5, 1],
                      [37.5, 0],
                      [42.5, 0],
                      [47.5, 0],
                      [52.5, 0],
                      [57.5, 0],
                      [62.5, 0],
                      [67.5, 0],
                      [72.5, 0],
                      [77.5, 0],
                      [82.5, 0],
                      [87.5, 0],
                      [92.5, 0],
                      [97.5, 0]],
                     [[2.5, 115],
                      [7.5, 107],
                      [12.5, 35],
                      [17.5, 5],
                      [2

In [258]:
# test one example for it AF info
test_AF = r.json().get("allele_freq")
test_AF

0.4883720930232558

In [229]:
# Use the query string in loop to query the Allele frequency of variant
# Someimtes the ExAC API has timeout, so we use a list incase the query loop is interrupted
AF_ExAC =[] 
for row in range(len(data)):
    query_name = 'http://exac.hms.harvard.edu/rest/variant/variant/'+report.loc[row]['Query_name']
    r = requests.get(query_name)
    ExAC_AF.append((r.json().get("allele_freq")))
# If we want, we can query more info such as Gene Symblo, etc.

report['AF_ExAC']=AF_ExAC # add this new information as a new column

In [254]:
#give a final look before save it
report

,variant_type,CHROM,POS,ref_allele,variant_allele,ref_allele_count,variant_allele_count,variant_AF,variant_vs_ref%,DP,Query_name,AF_ExAC
0,Subtitution,1,931393,G,T,4029,95,0.000000,2.360000e+00,4124,1-931393-G-T,NaN
1,Subtitution,1,935222,C,A,480,652,0.666667,1.358300e+02,1134,1-935222-C-A,0.661111
2,Subtitution,1,1277533,T,C,0,786,1.000000,inf,786,1-1277533-T-C,0.997571
3,Subtitution,1,1284490,G,A,0,228,1.000000,inf,228,1-1284490-G-A,0.897299
4,Subtitution,1,1571850,G,A,3961,94,0.000000,2.370000e+00,4055,1-1571850-G-A,0.000011
5,Subtitution,1,1572579,A,G,3430,26,0.000000,7.600000e-01,3456,1-1572579-A-G,0.002177
6,Subtitution,1,1575616,T,C,0,1118,1.000000,inf,1118,1-1575616-T-C,0.843984
7,Subtitution,1,1575784,C,T,3206,266,0.333333,8.300000e+00,3476,1-1575784-C-T,0.094886
8,Subtitution,1,1577180,C,T,40,50,0.666667,1.250000e+02,90,1-1577180-C-T,0.488372
9,Subtitution,1,1635004,T,C,0,4076,1.000000,inf,4076,1-1635004-T-C,0.580487


In [255]:
# save the final report as csv
export_csv = report.to_csv (r'annoted_VCF.csv', index = None, header=True)